BiRefNet HR Matting
====

**Bilateral Reference for High-Resolution Dichotomous Image Segmentation**

 * Paper: https://arxiv.org/abs/2401.03407

![BiRefNet Arch](../assets/birefnet_arch.png)

![BiRefNet Pipeline](../assets/birefnet_pipeline.png)

![BiRefNet Pipeline](../assets/birefnet_block.png)


```bash
conda create -n birefnet python=3.10 -y
conda activate birefnet

pip install -r https://raw.githubusercontent.com/ZhengPeng7/BiRefNet/main/requirements.txt

pip install transformers
pip install matplotlib
```